In [1]:
import json
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('formula1-originalDB.db')

In [3]:
query = '''
    SELECT
        driverId,
        forename,
        surname,
        dob,
        nationality
    FROM
        drivers
'''

drivers = pd.read_sql_query(query, conn)

print(drivers)

     driverId  forename     surname         dob nationality
0           1     Lewis    Hamilton  1985-01-07     British
1           2      Nick    Heidfeld  1977-05-10      German
2           3      Nico     Rosberg  1985-06-27      German
3           4  Fernando      Alonso  1981-07-29     Spanish
4           5    Heikki  Kovalainen  1981-10-19     Finnish
..        ...       ...         ...         ...         ...
852       854      Mick  Schumacher  1999-03-22      German
853       855    Guanyu        Zhou  1999-05-30     Chinese
854       856      Nyck    de Vries  1995-02-06       Dutch
855       857     Oscar     Piastri  2001-04-06  Australian
856       858     Logan    Sargeant  2000-12-31    American

[857 rows x 5 columns]


In [4]:
import sqlite3
import pandas as pd
import json

# Connect to the SQLite database
conn = sqlite3.connect('formula1-originalDB.db')  # Update with your actual database name

query = '''
    SELECT
        driverId,
        forename,
        surname,
        dob,
        nationality
    FROM
        drivers
'''

drivers = pd.read_sql_query(query, conn)

print(drivers)

# Query to calculate the maximum lap time for each driver in each season
max_query = '''
    SELECT
        drivers.driverId,
        strftime('%Y', races.date) AS season,
        MAX(lap_times.milliseconds) AS max_lap_time
    FROM
        lap_times
    JOIN
        races ON lap_times.raceId = races.raceId
    JOIN
        drivers ON lap_times.driverId = drivers.driverId
    GROUP BY
        drivers.driverId, season
'''

# Query to calculate the average lap time for each driver in each season
avg_query = '''
    SELECT
        drivers.driverId,
        strftime('%Y', races.date) AS season,
        AVG(lap_times.milliseconds) AS avg_lap_time
    FROM
        lap_times
    JOIN
        races ON lap_times.raceId = races.raceId
    JOIN
        drivers ON lap_times.driverId = drivers.driverId
    GROUP BY
        drivers.driverId, season
'''

# Query to calculate the minimum lap time for each driver in each season
min_query = '''
    SELECT
        drivers.driverId,
        strftime('%Y', races.date) AS season,
        MIN(lap_times.milliseconds) AS min_lap_time
    FROM
        lap_times
    JOIN
        races ON lap_times.raceId = races.raceId
    JOIN
        drivers ON lap_times.driverId = drivers.driverId
    GROUP BY
        drivers.driverId, season
'''

max_lap_times_df = pd.read_sql_query(max_query, conn)
avg_lap_times_df = pd.read_sql_query(avg_query, conn)
min_lap_times_df = pd.read_sql_query(min_query, conn)

# Create JSON columns for max, avg, and min lap times
max_lap_times_df['max_lap_times_json'] = max_lap_times_df.apply(
    lambda row: json.dumps({row['season']: row['max_lap_time']}), axis=1
)

avg_lap_times_df['avg_lap_times_json'] = avg_lap_times_df.apply(
    lambda row: json.dumps({row['season']: row['avg_lap_time']}), axis=1
)

min_lap_times_df['min_lap_times_json'] = min_lap_times_df.apply(
    lambda row: json.dumps({row['season']: row['min_lap_time']}), axis=1
)

# Group by driverId and aggregate the JSON dictionaries
max_lap_times_json_df = max_lap_times_df.groupby('driverId')['max_lap_times_json'].apply(list).reset_index()
avg_lap_times_json_df = avg_lap_times_df.groupby('driverId')['avg_lap_times_json'].apply(list).reset_index()
min_lap_times_json_df = min_lap_times_df.groupby('driverId')['min_lap_times_json'].apply(list).reset_index()

# Merge the aggregated JSON columns back into the 'drivers' DataFrame
drivers = pd.merge(drivers, max_lap_times_json_df, on='driverId', how='left')
drivers = pd.merge(drivers, avg_lap_times_json_df, on='driverId', how='left')
drivers = pd.merge(drivers, min_lap_times_json_df, on='driverId', how='left')

# Rename the columns and fill missing values with an empty list
drivers.rename(columns={'max_lap_times_json': 'max_lap_times', 'avg_lap_times_json': 'avg_lap_times', 'min_lap_times_json': 'min_lap_times'}, inplace=True)
drivers['max_lap_times'].fillna(value='[]', inplace=True)
drivers['avg_lap_times'].fillna(value='[]', inplace=True)
drivers['min_lap_times'].fillna(value='[]', inplace=True)

del min_lap_times_json_df, max_lap_times_json_df, avg_lap_times_json_df

# Display the resulting DataFrame
print(drivers)

     driverId  forename     surname         dob nationality
0           1     Lewis    Hamilton  1985-01-07     British
1           2      Nick    Heidfeld  1977-05-10      German
2           3      Nico     Rosberg  1985-06-27      German
3           4  Fernando      Alonso  1981-07-29     Spanish
4           5    Heikki  Kovalainen  1981-10-19     Finnish
..        ...       ...         ...         ...         ...
852       854      Mick  Schumacher  1999-03-22      German
853       855    Guanyu        Zhou  1999-05-30     Chinese
854       856      Nyck    de Vries  1995-02-06       Dutch
855       857     Oscar     Piastri  2001-04-06  Australian
856       858     Logan    Sargeant  2000-12-31    American

[857 rows x 5 columns]
     driverId  forename     surname         dob nationality  \
0           1     Lewis    Hamilton  1985-01-07     British   
1           2      Nick    Heidfeld  1977-05-10      German   
2           3      Nico     Rosberg  1985-06-27      German   
3   

In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('formula1-originalDB.db')

# SQL query
sql_query = '''
SELECT
    l.driverId,
    strftime('%Y', r.date) AS year,
    mode() WITHIN GROUP (ORDER BY l.position) AS most_common_position
FROM
    lap_times l
JOIN
    races r ON l.raceId = r.raceId
WHERE
    l.driverId IS NOT NULL
GROUP BY
    l.driverId,
    year
ORDER BY
    year,
    l.driverId;
'''

# Execute query and store result in a DataFrame
df = pd.read_sql_query(sql_query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


DatabaseError: Execution failed on sql '
SELECT
    l.driverId,
    strftime('%Y', r.date) AS year,
    mode() WITHIN GROUP (ORDER BY l.position) AS most_common_position
FROM
    lap_times l
JOIN
    races r ON l.raceId = r.raceId
WHERE
    l.driverId IS NOT NULL
GROUP BY
    l.driverId,
    year
ORDER BY
    year,
    l.driverId;
': near "(": syntax error